В архиве находятся три файла под названиями data.csv, price.csv,
quantity.csv. Первый содержит общую базу деталей поставщика, второй –
цены, третий - наличие.
Необходимо написать скрипт, который обработает эти 3 файла обьъединив
их так, чтобы в итоговом файле были только строки с наличием и ценой
больше нуля. Позиция из прайс листа считается корректной если во всех
столбцах есть данные.
Готовый файл должен быть в формате (part_number, manufacturer, price,
quantity).
Обработка должна быть реализована при помощи библиотеки Pandas.

In [3]:
import pandas as pd

In [8]:
data = pd.read_csv('data.csv', sep='\t')
prices = pd.read_csv('prices.csv', sep='\t')
quantity = pd.read_csv('quantity.csv', header=None, sep='\t')

print(data, prices, quantity)

       part_number     manufacturer
0          226 920            LINEX
1          039 042          SUNSTAR
2          039 056          SUNSTAR
3          039 063          SUNSTAR
4          051 893          KARCHER
...            ...              ...
477561     928 466    KOLBENSCHMIDT
477562     B13 757              ACI
477563     B13 788              ACI
477564     023 109        ZWM CZAJA
477565     167 240  MAGNETI MARELLI

[477566 rows x 2 columns]        part_number     price
0          000 002     54,36
1          000 004   5948,21
2          000 007  15999,74
3          000 008    225,01
4          000 014      8,80
...            ...       ...
467653     P01 345    254,70
467654     P01 346    386,67
467655     P01 347    682,90
467656     P01 349     61,83
467657     P01 350    234,06

[467658 rows x 2 columns]                0    1
0        000 002    1
1        000 008    1
2        000 014  >10
3        000 014    8
4        000 014    7
...          ...  ...
1048571  629

In [7]:
quantity_2 = quantity.rename(columns={0 : 'part_number', 1 : 'quantity'})
quantity_2['part_number'] = quantity_2['part_number'].astype(str)
quantity_2['quantity'] = quantity_2['quantity'].astype(str)
quantity_2

,part_number,quantity
0,000 002,1
1,000 008,1
2,000 014,>10
3,000 014,8
4,000 014,7
...,...,...
1048571,629 464,1
1048572,629 464,1
1048573,629 464,1
1048574,629 464,1


Здесь возникли проблемы с слиянием двух таблиц. Ренеймнул колонки "0" и "1" в "part_number" и "quantity", чтобы можно было выполнить мерж.

In [8]:
merged_data = pd.merge(data, prices, left_on='part_number', right_on='part_number', how='inner')
merged_data

,part_number,manufacturer,price
0,226 920,LINEX,"28,20"
1,039 042,SUNSTAR,"50,51"
2,039 056,SUNSTAR,"53,57"
3,051 893,KARCHER,"1050,13"
4,501 126,EPS,"28,66"
...,...,...,...
467950,928 466,KOLBENSCHMIDT,"77,03"
467951,B13 757,ACI,"10,81"
467952,B13 788,ACI,"68,79"
467953,023 109,ZWM CZAJA,"5,18"


In [44]:
remerged_data = pd.merge(merged_data, quantity_2, left_on='part_number', right_on='part_number', how='inner')
remerged_data

,part_number,manufacturer,price,quantity
0,226 920,LINEX,"28,20",>10
1,226 920,LINEX,"28,20",1
2,226 920,LINEX,"28,20",2
3,226 920,LINEX,"28,20",>10
4,039 042,SUNSTAR,"50,51",1
...,...,...,...,...
1051556,023 109,ZWM CZAJA,"5,18",1
1051557,023 109,ZWM CZAJA,"5,18",1
1051558,023 109,ZWM CZAJA,"5,18",1
1051559,023 109,ZWM CZAJA,"5,18",1


In [45]:
remerged_data['quantity'] = remerged_data['quantity'].str.replace('>', '')
remerged_data['quantity'] = remerged_data['quantity'].astype(int)
remerged_data['price'] = remerged_data['price'].str.replace(',', '.')
remerged_data['price'] = remerged_data['price'].astype(float)
filtered_data = remerged_data[(remerged_data['quantity'] > 0) & (remerged_data['price'] > 0)]
filtered_data

,part_number,manufacturer,price,quantity
0,226 920,LINEX,28.20,10
1,226 920,LINEX,28.20,1
2,226 920,LINEX,28.20,2
3,226 920,LINEX,28.20,10
4,039 042,SUNSTAR,50.51,1
...,...,...,...,...
1051556,023 109,ZWM CZAJA,5.18,1
1051557,023 109,ZWM CZAJA,5.18,1
1051558,023 109,ZWM CZAJA,5.18,1
1051559,023 109,ZWM CZAJA,5.18,1


Тут возникли проблемы при сравнивании. Обе колонки были стринговые, но чтобы их певерести в int и float, нужно убрать знак ">" в количестве и заменить запятую на точку в колонке прайс.

Также необходимо реализовать метод, который будет предоставлять отчет
о колличестве брендов и позиций под ними в формате БРЕНД1 — 5000
rows и т. д.

In [108]:
brand_counts = filtered_data.groupby('manufacturer').size().reset_index(name='Counter')
brand_counts = brand_counts.sort_values(by='Counter', ascending=False)
brand_counts = brand_counts.reset_index(drop=True)
brand_counts

,manufacturer,Counter
0,HART,255466
1,BOSCH,56774
2,FEBI BILSTEIN,50580
3,SIDEM,35672
4,FILTRON,29360
...,...,...
493,PRONAR,1
494,CTEK,1
495,VANKING CELKAR,1
496,VARIANT SERVICE,1


In [109]:
save_1 = filtered_data.to_csv('filtered_data.csv', index=False, sep='\t')
save_2 = brand_counts.to_csv('brand_counts.csv', index=False, sep='\t')